In [ ]:
import h2o
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
h2o.init()

In [ ]:
## creating a h2o data frame
df = pd.read_excel('../input/Dataset.xlsx')

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
df.isnull().sum()
df=df.drop('Baseline',axis=1)
df=df.drop('During operation',axis=1)
df=df.drop('First CT',axis=1)
df=df.drop('Follow-up',axis=1)
df=df.drop('MV duration:h',axis=1)
df=df.drop('Endotracheal duration:h',axis=1)
df=df.drop('ReIntubation',axis=1)
df.head()

In [ ]:
df.isnull().sum()
df=df.drop('At ICU admission',axis=1)


In [ ]:
df.isnull().sum()
sns.heatmap(df.corr())

## converting to h2o df

In [ ]:
dfh = h2o.H2OFrame(df)

In [ ]:
dfh.head()

In [ ]:
type(dfh)

In [ ]:
df[['Agitation']].hist()

In [ ]:
## too much biased target

In [ ]:
df.describe()

In [ ]:
train_h,test_h,valid_h = dfh.split_frame([0.6,0.2])

In [ ]:
dfh.as_data_frame().plot()

In [ ]:
y = 'Agitation'

In [ ]:
x = dfh.names

In [ ]:
x.remove(y)

In [ ]:
from h2o.automl import H2OAutoML

In [ ]:
aml = H2OAutoML(max_models=25, seed=1)
aml.train(x=x, y=y, training_frame=train_h)

In [ ]:
lb = aml.leaderboard
lb


In [ ]:
preds = aml.leader.predict(test_h)

In [ ]:
preds

# DEEP LEARNING

In [ ]:
from h2o.estimators import deeplearning
m = h2o.estimators.deeplearning.H2ODeepLearningEstimator()
m.train(x, y, train_h)
p = m.predict(test_h)

In [ ]:
r2 = m.r2()
mse = m.mse()
rmse = m.rmse()

In [ ]:
mse

In [ ]:
rmse

In [ ]:
r2

In [ ]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [ ]:
model = H2ODeepLearningEstimator(
activation="RectifierWithDropout",
hidden=[128,128,128,128],
input_dropout_ratio=0.2,
sparse=True,
epochs=10)

In [ ]:
model.train(
x=x,
y=y,
training_frame=train_h,
validation_frame=test_h)

In [ ]:
model

In [ ]:
model.predict(test_h)

In [ ]:
## DEEP LEARNING WITH GRID SEARCH

In [ ]:
hidden_opt = [[32,32],[32,16,8],[100]]
#l1_opt = [1e-4,1e-3]
hyper_parameters = {"hidden":hidden_opt}
from h2o.grid.grid_search import H2OGridSearch
model_grid = H2OGridSearch(H2ODeepLearningEstimator,
hyper_params=hyper_parameters)
model_grid.train(x=x, y=y, epochs=1000,
training_frame=train_h, validation_frame=test_h,
score_interval=2, stopping_rounds=3,
stopping_tolerance=0.05)

In [ ]:
model_grid

In [ ]:
model_grid.r2

In [ ]:
model_grid.mse()